# [Basic Models]

*KU LeeDongGyu*

### Module

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import backend as K

### F1-score Function

In [2]:
'''
아래 코드는 f1-score를 정의한 함수. 
tensorflow2.0 버전 이후로 f1-score 지원이 metrics애서 사라졌다.
'''

def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1))  # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))  # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn)
    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)
    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())
    # return a single tensor value
    return recall

def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))  # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1))  # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn)
    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)
    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())
    # return a single tensor value
    return precision

def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = (2 * _recall * _precision) / (_recall + _precision + K.epsilon())
    # return a single tensor value
    return _f1score


## 1. MLP
---

In [3]:
###################### 1. model - MLP ######################

#data import
x_train = pd.read_csv("C:\\Users\\82104\\Desktop\\fer2013\\mydata\\X_train.csv",
                      header=0,index_col=0)
x_valid = pd.read_csv("C:\\Users\\82104\\Desktop\\fer2013\\mydata\\X_private_test.csv",
                      header=0,index_col=0)
x_test = pd.read_csv("C:\\Users\\82104\\Desktop\\fer2013\\mydata\\X_public_test.csv",
                      header=0,index_col=0)
y_train = pd.read_csv("C:\\Users\\82104\\Desktop\\fer2013\\mydata\\y_train.csv",
                      header=0,index_col=0)
y_valid = pd.read_csv("C:\\Users\\82104\\Desktop\\fer2013\\mydata\\y_private_test.csv",
                      header=0,index_col=0)
y_test = pd.read_csv("C:\\Users\\82104\\Desktop\\fer2013\\mydata\\y_public_test.csv",
                      header=0,index_col=0)

In [4]:
# data handling
x_train = np.array(x_train).reshape([-1,48,48]) / 255
x_valid = np.array(x_valid).reshape([-1,48,48]) / 255
x_test = np.array(x_test).reshape([-1,48,48]) / 255
y_train = np.array(y_train).reshape([-1,])
y_valid = np.array(y_valid).reshape([-1,])
y_test = np.array(y_test).reshape([-1,])

In [5]:
epochs = 15
classes = len(np.unique(y_test))
batch_size=128

In [6]:
# MLP model
inputs = tf.keras.Input(shape=(48,48))
x=tf.keras.layers.Flatten()(inputs)
x=tf.keras.layers.Dense(units=128,activation = 'relu',name='d1')(x)
x=tf.keras.layers.Dropout(0.3)(x)
x=tf.keras.layers.Dense(units=512,activation = 'relu',name='d2')(x)
x=tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(units=classes,activation = tf.nn.softmax,name='d3')(x)


In [7]:
model = tf.keras.Model(inputs=inputs,outputs=outputs)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48)]          0         
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
d1 (Dense)                   (None, 128)               295040    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
d2 (Dense)                   (None, 512)               66048     
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
d3 (Dense)                   (None, 7)                 3591  

In [9]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',metrics=['accuracy',f1score])


In [10]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=epochs)

Train on 28698 samples, validate on 3589 samples
Epoch 1/15
28698/28698 [==============================] - 4s 128us/sample - loss: 1.8227 - accuracy: 0.2439 - f1score: 0.0149 - val_loss: 1.8042 - val_accuracy: 0.2449 - val_f1score: 0.0000e+00
Epoch 2/15
28698/28698 [==============================] - 3s 97us/sample - loss: 1.7862 - accuracy: 0.2509 - f1score: 0.0585 - val_loss: 1.7906 - val_accuracy: 0.2449 - val_f1score: 0.0056
Epoch 3/15
28698/28698 [==============================] - 3s 97us/sample - loss: 1.7893 - accuracy: 0.2466 - f1score: 0.0678 - val_loss: 1.7689 - val_accuracy: 0.2449 - val_f1score: 0.0174
Epoch 4/15
28698/28698 [==============================] - 3s 89us/sample - loss: 1.7754 - accuracy: 0.2497 - f1score: 0.1006 - val_loss: 1.7569 - val_accuracy: 0.2792 - val_f1score: 0.0318
Epoch 5/15
28698/28698 [==============================] - 3s 92us/sample - loss: 1.7716 - accuracy: 0.2522 - f1score: 0.1160 - val_loss: 1.7476 - val_accuracy: 0.2792 - val_f1score: 0.0336
E

MLP <br>
In Epoch 8/15
<br> 
Train // $\quad$ Accuracy : 0.2534 $\quad$ F1 Score : 0.1226  <br>
Valid //  $\quad$ Accuracy : 0.2527 $\quad$  F1 Score : 0.0289 

In [2]:
#_, acc, f1 = model.evaluate(x_test,y_test, batch_size=batch_size) # early stopping으로 training에서  epoch 8번후 적용해야함.
#print("\nAccuracy: {:.4f}, F1 Score: {:.4f}".format(acc,f1))

## 2. Basic CNN
---

In [12]:
###################### 2. model - CNN Basic ######################

#data import
x_train = pd.read_csv("C:\\Users\\82104\\Desktop\\fer2013\\mydata\\X_train.csv",
                      header=0,index_col=0)
x_valid = pd.read_csv("C:\\Users\\82104\\Desktop\\fer2013\\mydata\\X_private_test.csv",
                      header=0,index_col=0)
x_test = pd.read_csv("C:\\Users\\82104\\Desktop\\fer2013\\mydata\\X_public_test.csv",
                      header=0,index_col=0)
y_train = pd.read_csv("C:\\Users\\82104\\Desktop\\fer2013\\mydata\\y_train.csv",
                      header=0,index_col=0)
y_valid = pd.read_csv("C:\\Users\\82104\\Desktop\\fer2013\\mydata\\y_private_test.csv",
                      header=0,index_col=0)
y_test = pd.read_csv("C:\\Users\\82104\\Desktop\\fer2013\\mydata\\y_public_test.csv",
                      header=0,index_col=0)

In [13]:
#data handling
x_train = np.array(x_train).reshape([-1,48,48,1]) / 255
x_valid = np.array(x_valid).reshape([-1,48,48,1]) / 255
x_test = np.array(x_test).reshape([-1,48,48,1]) / 255
y_train = np.array(y_train).reshape([-1,])
y_valid = np.array(y_valid).reshape([-1,])
y_test = np.array(y_test).reshape([-1,])

In [14]:
epochs = 15
image_size=48
classes = len(np.unique(y_test))

input_shape = (image_size,image_size,1)
batch_size = 128
kernel_size = (3,3) 
filters = 64 
dropout = 0.3 

In [15]:
# CNN model
cnn_model = tf.keras.models.Sequential()
cnn_model.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size,
                                     activation='relu', input_shape=input_shape, strides = (1,1) , name='Conv2D_layer1'))
cnn_model.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling1_2D'))
cnn_model.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size,
                                     activation='relu', input_shape=input_shape, strides = (1,1) , name='Conv2D_layer2'))
cnn_model.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling2_2D'))
cnn_model.add(tf.keras.layers.Flatten(name='Flatten'))
cnn_model.add(tf.keras.layers.Dropout(dropout))
cnn_model.add(tf.keras.layers.Dense(64, activation='relu', name='Hidden_layer'))
cnn_model.add(tf.keras.layers.Dense(classes, activation='softmax', name='Output_layer'))

In [16]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2D_layer1 (Conv2D)       (None, 46, 46, 64)        640       
_________________________________________________________________
Maxpooling1_2D (MaxPooling2D (None, 23, 23, 64)        0         
_________________________________________________________________
Conv2D_layer2 (Conv2D)       (None, 21, 21, 64)        36928     
_________________________________________________________________
Maxpooling2_2D (MaxPooling2D (None, 10, 10, 64)        0         
_________________________________________________________________
Flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6400)              0         
_________________________________________________________________
Hidden_layer (Dense)         (None, 64)                4

In [17]:
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
                  metrics=['accuracy',f1score])

In [18]:
cnn_model.fit(x_train,y_train, validation_data=(x_valid,y_valid) ,batch_size=batch_size,epochs=epochs)

Train on 28698 samples, validate on 3589 samples
Epoch 1/15
28698/28698 [==============================] - 118s 4ms/sample - loss: 1.7035 - accuracy: 0.3190 - f1score: 0.0947 - val_loss: 1.5898 - val_accuracy: 0.3906 - val_f1score: 0.1948
Epoch 2/15
28698/28698 [==============================] - 113s 4ms/sample - loss: 1.5358 - accuracy: 0.4096 - f1score: 0.3287 - val_loss: 1.4740 - val_accuracy: 0.4291 - val_f1score: 0.4406
Epoch 3/15
28698/28698 [==============================] - 115s 4ms/sample - loss: 1.4441 - accuracy: 0.4491 - f1score: 0.4093 - val_loss: 1.4419 - val_accuracy: 0.4508 - val_f1score: 0.4819
Epoch 4/15
28698/28698 [==============================] - 116s 4ms/sample - loss: 1.3824 - accuracy: 0.4731 - f1score: 0.4677 - val_loss: 1.3511 - val_accuracy: 0.4781 - val_f1score: 0.4560
Epoch 5/15
28698/28698 [==============================] - 113s 4ms/sample - loss: 1.3374 - accuracy: 0.4925 - f1score: 0.5087 - val_loss: 1.3466 - val_accuracy: 0.4778 - val_f1score: 0.5495
E

Basic CNN <br>
In Epoch 6/15
<br> 
Train // $\quad$ Accuracy : 0.5098 $\quad$ F1 Score : 0.5382  <br>
Valid //  $\quad$ Accuracy : 0.5038 $\quad$  F1 Score : 0.5375 

In [1]:
#_, acc, f1 = cnn_model.evaluate(x_test,y_test,batch_size=batch_size) # early stopping으로 training에서 epoch 6번후 적용해야함.
#print("\nAccuracy: {:.4f}, F1 Score: {:.4f}".format(acc,f1))